In [147]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import math
import numpy as np

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("KEY")

In [4]:
def get_results_from_foursquare (query, location, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={location[0]}%2C{location[1]}&sort=DISTANCE&{limit}=5"
    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"]

In [5]:
query = "cafe"
Ironhack_lat=41.3977737
Ironhack_lon=2.1882674
location = [Ironhack_lat, Ironhack_lon]
limit = 10
results = get_results_from_foursquare (query, location, limit)

In [32]:
client = MongoClient("localhost:27017")
db = client.get_database("Ironhack")

In [57]:
companies = db.get_collection("companies")

In [90]:
valencia_companies = list(companies.find({"offices.city":"Valencia"}))
barcelona_companies = list(companies.find({"offices.city":"Barcelona"}))
madrid_companies = list(companies.find({"offices.city":"Madrid"}))
spain_companies = [valencia_companies, barcelona_companies, madrid_companies]

In [116]:
name = []
city = []
office_number = []
address = []
latitude = []
longitude = []
for each_city in spain_companies:
    for company in each_city:
        try:
            for i in range(len(company["offices"])):
                n = 0
                if company["offices"][i]["city"] in ["Valencia", "Barcelona", "Madrid"]:
                    n += 1
                    name.append(company["name"])
                    city.append(company["offices"][i]["city"])
                    office_number.append(n)
                    address.append(company["offices"][i]["address1"])
                    latitude.append(company["offices"][i]["latitude"])
                    longitude.append(company["offices"][i]["longitude"])
        except:
            pass

In [125]:
spain_dict = {"name":name, "city":city, "office_number":office_number, "address":address, "latitude":latitude, "longitude":longitude}

In [126]:
df = pd.DataFrame.from_dict(spain_dict)

In [148]:
df['address'].replace('', np.nan, inplace=True)

In [152]:
df = df.dropna(subset=["address", "latitude", "longitude"], how='all')

In [161]:
def get_coordinates (place):
    try:
        res = requests.get(f"https://geocode.xyz/{place}?json=1").json()
        return [res["longt"],res["latt"]]
    except:
        return [np.nan, np.nan]

In [ ]:
for i in df.index.values.tolist():
    if math.isnan(df.at[i, "latitude"]) or df.at[i, "latitude"]==0:
        address_complete = df.at[i, "address"] + ", " + df.at[i, "city"] + ", Spain"
        coordinates = get_coordinates(address_complete)
        df.at[i, "latitude"]= float(coordinates[1])
        df.at[i, "longitude"]= float(coordinates[0])

In [196]:
for i in df.index.values.tolist():
    if df.at[i, "latitude"] < 20:
        latitude = df.at[i, "latitude"]
        longitude = df.at[i, "longitude"]
        df.at[i, "latitude"] = longitude
        df.at[i, "longitude"] = latitude

In [198]:
df['latitude'] = df['latitude'].fillna(0)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\3028638243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['latitude'].fillna(0)


In [200]:
df.drop(df[df.latitude < 30].index, inplace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\1358260333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df.latitude < 30].index, inplace=True)


In [209]:
df.drop(df[df.longitude < -10].index, inplace=True)

In [210]:
df.to_csv("Output/spanish_companies.csv")

In [211]:
df = pd.read_csv('Output/spanish_companies.csv')

In [212]:
df = df.set_index('Unnamed: 0')
df.index.name = None

In [244]:
position = ["Designer", "UI/UX Engineers", "Frontend Developers", "Data Engineers", 
             "Backend Developers", "Account Managers", "Maintenance", "Executives", "CEO"]
number_of_employees = [20, 5, 10, 15, 5, 20, 1, 10, 1]
salary = [30000, 33000, 33000, 44000, 39000, 25000, 20000, 60000, 80000]

In [245]:
employees_dict = {"position":position, "number_of_employees":number_of_employees, "salary":salary}

In [246]:
df2 = pd.DataFrame.from_dict(employees_dict)

In [247]:
df2

,position,number_of_employees,salary
0,Designer,20,30000
1,UI/UX Engineers,5,33000
2,Frontend Developers,10,33000
3,Data Engineers,15,44000
4,Backend Developers,5,39000
5,Account Managers,20,25000
6,Maintenance,1,20000
7,Executives,10,60000
8,CEO,1,80000


In [267]:
df2[df2['position'] == "Designer"].at[0, "number_of_employees"]

20

In [256]:
list(df2["position"].values)

['Designer',
 'UI/UX Engineers',
 'Frontend Developers',
 'Data Engineers',
 'Backend Developers',
 'Account Managers',
 'Maintenance',
 'Executives',
 'CEO']

In [317]:
requirements = ["Design company", "Schools", "Successful tech startup", "Starbucks", "Airport", "Party", 
                "Vegan restaurant", "Basketball stadium", "Dog hairdresser"]
related_employees = ["Designer", "30% of All", "Frontend Developers/Backend Developers", "Executives", "Account Managers", "All", 
                     "CEO", "Maintenance", "All"]
weight = [5, 10, 5, 4, 8, 2, 4, 4, 1]

In [318]:
requirements_dict = {"requirements":requirements, "related_employees":related_employees, "weight":weight}

In [319]:
df3 = pd.DataFrame.from_dict(requirements_dict)

In [320]:
df3

,requirements,related_employees,weight
0,Design company,Designer,5
1,Schools,30% of All,10
2,Successful tech startup,Frontend Developers/Backend Developers,5
3,Starbucks,Executives,4
4,Airport,Account Managers,8
5,Party,All,2
6,Vegan restaurant,CEO,4
7,Basketball stadium,Maintenance,4
8,Dog hairdresser,All,1


In [321]:
final_weight = []
everyone = list(df2["position"].values)
related_employees = [["Designer"], everyone, ['Frontend Developers', 'Backend Developers'], ['Executives'], ['Account Managers'], everyone, ['CEO'], ['Maintenance'], everyone] 
for i, employee_types in enumerate(related_employees):
    weight = df3.at[i, "weight"]
    final = 0
    for employee_type in employee_types:
        number_of_employees = list(df2.loc[df2['position'] == employee_type]["number_of_employees"].values)[0]
        salary = list(df2.loc[df2['position'] == employee_type]["salary"].values)[0]
        final += weight*number_of_employees*salary
    if i == 1: 
        final *= 0.3
    final_weight.append(int(final))

In [322]:
final_weight

[3000000, 9450000, 2625000, 2400000, 4000000, 6300000, 320000, 80000, 3150000]

In [323]:
df3['final_weight'] = final_weight

In [324]:
df3

,requirements,related_employees,weight,final_weight
0,Design company,Designer,5,3000000
1,Schools,30% of All,10,9450000
2,Successful tech startup,Frontend Developers/Backend Developers,5,2625000
3,Starbucks,Executives,4,2400000
4,Airport,Account Managers,8,4000000
5,Party,All,2,6300000
6,Vegan restaurant,CEO,4,320000
7,Basketball stadium,Maintenance,4,80000
8,Dog hairdresser,All,1,3150000


In [ ]:
"""
CHANGE WEIGHTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS
"""

In [ ]:
!git add my